In [1]:
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os
from scipy.io import mmwrite
import desc
import scipy
import anndata as ad

In [2]:
sc_combine=sc.read_h5ad("/home/huggs/shiyi/SCST/RCTD/RCTD/predata/celltrek_kidney/mymodel/2_desc/2_desc_222copy/sc_desc.h5ad")
sc_combine.X=sc_combine.raw.X
sc_combine.obs["batch"]="sc"
sc_combine

AnnData object with n_obs × n_vars = 40712 × 2977
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'n_counts', 'desc_0.2', 'desc_0.4', 'desc_0.6', 'desc_0.8', 'desc_1.4', 'desc_2.0', 'batch'
    var: 'features', 'mean', 'std'
    uns: 'prob_matrix0.2', 'prob_matrix0.4', 'prob_matrix0.6', 'prob_matrix0.8', 'prob_matrix1.4', 'prob_matrix2.0', 'umap'
    obsm: 'X_Embeded_z0.2', 'X_Embeded_z0.4', 'X_Embeded_z0.6', 'X_Embeded_z0.8', 'X_Embeded_z1.4', 'X_Embeded_z2.0', 'X_tsne', 'X_tsne0.2', 'X_tsne0.4', 'X_tsne0.6', 'X_tsne0.8', 'X_tsne1.4', 'X_tsne2.0', 'X_umap', 'X_umap0.2', 'X_umap0.4', 'X_umap0.6', 'X_umap0.8', 'X_umap1.4', 'X_umap2.0'
    obsp: 'connectivities', 'distances'

In [3]:
st_combine=sc.read_h5ad("/home/huggs/shiyi/SCST/RCTD/RCTD/predata/celltrek_kidney/mymodel/1_predata/st_combine.h5ad")
st_combine.obs["batch"]="st"
st_combine

AnnData object with n_obs × n_vars = 1438 × 2977
    obs: 'orig.ident', 'nCount_Spatial', 'nFeature_Spatial', 'batch'
    var: 'features'

In [4]:
#合并ST,adata
import anndata as ad
adatas=[sc_combine,st_combine]
adatas=ad.concat(adatas,merge='same')
adatas

AnnData object with n_obs × n_vars = 42150 × 2977
    obs: 'orig.ident', 'batch'
    var: 'features'

In [5]:
#combine try 2
import anndata as ad
combined = ad.concat([sc_combine,st_combine], join="outer")
combined

AnnData object with n_obs × n_vars = 42150 × 2977
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'n_counts', 'desc_0.2', 'desc_0.4', 'desc_0.6', 'desc_0.8', 'desc_1.4', 'desc_2.0', 'batch', 'nCount_Spatial', 'nFeature_Spatial'
    obsm: 'X_Embeded_z0.2', 'X_Embeded_z0.4', 'X_Embeded_z0.6', 'X_Embeded_z0.8', 'X_Embeded_z1.4', 'X_Embeded_z2.0', 'X_tsne', 'X_tsne0.2', 'X_tsne0.4', 'X_tsne0.6', 'X_tsne0.8', 'X_tsne1.4', 'X_tsne2.0', 'X_umap', 'X_umap0.2', 'X_umap0.4', 'X_umap0.6', 'X_umap0.8', 'X_umap1.4', 'X_umap2.0'

In [6]:
combined.obs['desc_0.8'].astype('category')

Z1_F_F1_AAACCTGAGAGACGAA     16
Z1_F_F1_AAACCTGAGGGCTTCC     16
Z1_F_F1_AAACCTGCAGGACGTA      2
Z1_F_F1_AAACCTGGTATTCGTG      2
Z1_F_F1_AAACCTGTCAAACCAC      2
                           ... 
TTGTGTATGCCACCAA-1          NaN
TTGTGTTTCCCGAAAG-1          NaN
TTGTTCAGTGTGCTAC-1          NaN
TTGTTGTGTGTCAAGA-1          NaN
TTGTTTCACATCCAGG-1          NaN
Name: desc_0.8, Length: 42150, dtype: category
Categories (28, int64): [0, 1, 2, 3, ..., 24, 25, 26, 27]

In [7]:
combined.var=adatas.var

In [8]:
combined

AnnData object with n_obs × n_vars = 42150 × 2977
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'n_counts', 'desc_0.2', 'desc_0.4', 'desc_0.6', 'desc_0.8', 'desc_1.4', 'desc_2.0', 'batch', 'nCount_Spatial', 'nFeature_Spatial'
    var: 'features'
    obsm: 'X_Embeded_z0.2', 'X_Embeded_z0.4', 'X_Embeded_z0.6', 'X_Embeded_z0.8', 'X_Embeded_z1.4', 'X_Embeded_z2.0', 'X_tsne', 'X_tsne0.2', 'X_tsne0.4', 'X_tsne0.6', 'X_tsne0.8', 'X_tsne1.4', 'X_tsne2.0', 'X_umap', 'X_umap0.2', 'X_umap0.4', 'X_umap0.6', 'X_umap0.8', 'X_umap1.4', 'X_umap2.0'

In [9]:
del combined.obs['desc_0.4']
del combined.obs['desc_0.6']
del combined.obs['desc_0.2']
del combined.obs['desc_1.4']
del combined.obs['desc_2.0']

In [10]:
combined.obs['desc_0.8']=pd.Series(combined.obs['desc_0.8']).astype("category")
combined.write_h5ad("combine_allst.h5ad")

... storing 'batch' as categorical


In [11]:
#平均表达个数
scaverages=combined[combined.obs['batch']=='sc'].obs['nFeature_RNA'].mean() #sc_combine
print(scaverages)
staverages=combined[combined.obs['batch']=='st'].obs['nFeature_Spatial'].mean()   #st_combine
print(staverages)
alpha=scaverages/staverages 
alpha 
#231.72607584987227
#1121.7586926286508
#0.20657390700210365

231.72607584987227
1121.7586926286508


0.20657390700210365

In [ ]:
#下采样

In [12]:
def lower_sample_data(rawdata, alpha):
    """
    将矩阵中的每个元素有概率 p 保留原值，有概率 1-p 变成 0
    """
    #创建一个与原始矩阵大小相同的随机矩阵
    random_matrix=np.random.choice([0,1], size=(rawdata.shape[0],rawdata.shape[1]), replace=True, p=[1-alpha,alpha])
    # 将原始矩阵中的元素与随机矩阵中的元素按位相乘
    result = rawdata * random_matrix
    return result

In [13]:
traindatasubset_sc=combined[combined.obs['batch']=='sc']
traindatasubset_st=combined[combined.obs['batch']=='st']
traindatasubset_st

View of AnnData object with n_obs × n_vars = 1438 × 2977
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'n_counts', 'desc_0.8', 'batch', 'nCount_Spatial', 'nFeature_Spatial'
    var: 'features'
    obsm: 'X_Embeded_z0.2', 'X_Embeded_z0.4', 'X_Embeded_z0.6', 'X_Embeded_z0.8', 'X_Embeded_z1.4', 'X_Embeded_z2.0', 'X_tsne', 'X_tsne0.2', 'X_tsne0.4', 'X_tsne0.6', 'X_tsne0.8', 'X_tsne1.4', 'X_tsne2.0', 'X_umap', 'X_umap0.2', 'X_umap0.4', 'X_umap0.6', 'X_umap0.8', 'X_umap1.4', 'X_umap2.0'

In [14]:
lower_sample_data_1=lower_sample_data(pd.DataFrame(traindatasubset_st.X.todense()), alpha)
lower_sample_data_1
lower_sample_data_1.index=traindatasubset_st.obs_names
lower_sample_data_1.index
lower_sample_data_1.columns=traindatasubset_st.var_names
lower_sample_data_1.columns
traindata_st=sc.AnnData(lower_sample_data_1)
traindata_st
traindata_st.obs["batch"]="st"

In [15]:
int(traindatasubset_sc.shape[0]/traindatasubset_st.shape[0])

28

In [16]:
###循环抽样sc/st次
##desc_0.8
names = locals()
for i in range(1,int(traindatasubset_sc.shape[0]/traindatasubset_st.shape[0])+1):
    print(i)
    names['lower_sample_data_%s'%i]=lower_sample_data(pd.DataFrame(traindatasubset_st.X.todense()), alpha)
    names['lower_sample_data_%s'%i].index=traindatasubset_st.obs_names
    names['lower_sample_data_%s'%i].columns=traindatasubset_st.var_names
    names['traindata_st_%s'%i]=sc.AnnData(lower_sample_data_1)
    names['traindata_st_%s'%i].obs["batch"]="st"
    names['traindata_st_%s'%i].obs["desc_0.8"]=traindatasubset_st.obs['desc_0.8']
    names['traindata_st_%s'%i].obs["batch_sub"]='st_%s'%i
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [19]:
#sc_combine.X=sc_combine.X.toarray()

In [17]:
import anndata as ad
names = locals()
traindata_combine_1=traindata_st_1
for i in range(1,int(traindatasubset_sc.shape[0]/traindatasubset_st.shape[0])):
    print(i)
    names['traindata_combine_%s'%(i+1)]=ad.concat([names['traindata_combine_%s'%i],names['traindata_st_%s'%(i+1)]], join="outer")


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


1
2
3
4
5


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


6
7
8
9


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


...

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


26


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


27


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [18]:
traindata_combine_28

AnnData object with n_obs × n_vars = 40264 × 2977
    obs: 'batch', 'desc_0.8', 'batch_sub'

In [19]:
traindata_combine_28.obs['desc_0.8']=pd.Series(traindata_combine_28.obs['desc_0.8']).astype("category")

In [20]:
traindata_combine_28.write_h5ad("testdata_undersample_st.h5ad")

... storing 'batch' as categorical
... storing 'batch_sub' as categorical


In [21]:
traindata_combine=ad.concat([sc_combine,traindata_combine_28],join="outer")
traindata_combine

/home/huggs/anaconda3/envs/SCST/lib/python3.6/site-packages/anndata/_core/merge.py:918: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  UserWarning,
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


AnnData object with n_obs × n_vars = 80976 × 2977
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'n_counts', 'desc_0.2', 'desc_0.4', 'desc_0.6', 'desc_0.8', 'desc_1.4', 'desc_2.0', 'batch', 'batch_sub'
    obsm: 'X_Embeded_z0.2', 'X_Embeded_z0.4', 'X_Embeded_z0.6', 'X_Embeded_z0.8', 'X_Embeded_z1.4', 'X_Embeded_z2.0', 'X_tsne', 'X_tsne0.2', 'X_tsne0.4', 'X_tsne0.6', 'X_tsne0.8', 'X_tsne1.4', 'X_tsne2.0', 'X_umap', 'X_umap0.2', 'X_umap0.4', 'X_umap0.6', 'X_umap0.8', 'X_umap1.4', 'X_umap2.0'

In [22]:
del traindata_combine.obs['desc_0.4']
del traindata_combine.obs['desc_0.6']
del traindata_combine.obs['desc_0.2']
del traindata_combine.obs['desc_1.4']
del traindata_combine.obs['desc_2.0']
traindata_combine.obs['desc_0.8']=pd.Series(traindata_combine.obs['desc_0.8']).astype("category")
traindata_combine.write_h5ad("testdata_undersample.h5ad")

... storing 'batch' as categorical
... storing 'batch_sub' as categorical
